# 特征值选择实验项目

在机器学习项目中，特征选择是一个非常重要的步骤，它可以帮助提高模型的性能、减少计算成本，并提高模型的解释性。特征选择可以通过不同的方法进行，包括过滤法（Filter Methods）、包裹法（Wrapper Methods）和嵌入法（Embedded Methods）。本 Notebook 将介绍如何使用 Python 中的几种常见库（如 scikit-learn、pandas 和 numpy）来进行特征选择。

## 1. 准备环境

首先，确保你已经安装了必要的 Python 库。

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## 2. 加载数据

我们将使用 scikit-learn 库中的葡萄酒数据集作为示例。

In [2]:
# 加载数据
data = load_wine()
X = data.data
y = data.target
# 划分训练集和测试集，改变random_state以增加随机性
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)

## 3. 过滤法（Filter Methods）

过滤法通过计算每个特征与目标变量之间的统计相关性来选择特征。例如，可以使用卡方检验。

In [3]:
# 使用卡方检验选择最佳特征，改变k值
selector = SelectKBest(chi2, k=3)  # 选择前三个最佳特征
X_train_new_filter = selector.fit_transform(X_train, y_train)
X_test_new_filter = selector.transform(X_test)

## 4. 包裹法（Wrapper Methods）

包裹法通过训练一个模型来评估每个特征的重要性。例如，可以使用递归特征消除（RFE）或基于树的特征选择。

In [4]:
# 使用随机森林进行特征选择，改变n_features_to_select的值
model = RandomForestClassifier(n_estimators=100, random_state=43)
rfe = RFE(model, n_features_to_select=3)  # 选择前三个最佳特征
X_train_new_wrapper = rfe.fit_transform(X_train, y_train)
X_test_new_wrapper = rfe.transform(X_test)

## 5. 嵌入法（Embedded Methods）

嵌入法是将特征选择融入到模型训练过程中的方法，例如使用基于树的模型如随机森林或梯度提升树。这些模型在训练时会考虑特征的重要性。

In [5]:
# 使用梯度提升树进行特征选择，改变模型的超参数
model = GradientBoostingClassifier(n_estimators=50, random_state=43)
model.fit(X_train, y_train)
sfm = SelectFromModel(model, prefit=True)  # 使用模型选择特征
X_train_new_embedded = sfm.transform(X_train)
X_test_new_embedded = sfm.transform(X_test)

## 6. 评估模型性能

最后，使用选定的特征训练模型并评估其性能。

In [6]:
# 使用SVM进行分类并评估模型性能
# 过滤法
clf_filter = SVC(kernel='linear', C=1)
clf_filter.fit(X_train_new_filter, y_train)
y_pred_filter = clf_filter.predict(X_test_new_filter)
print("过滤法 — 准确率:", accuracy_score(y_test, y_pred_filter))
print("过滤法 — 分类报告:\n", classification_report(y_test, y_pred_filter, zero_division=0))
print("------------------------------------------------------------")
# 包裹法
clf_wrapper = SVC(kernel='rbf', C=1)
clf_wrapper.fit(X_train_new_wrapper, y_train)
y_pred_wrapper = clf_wrapper.predict(X_test_new_wrapper)
print("包裹法 — 准确率:", accuracy_score(y_test, y_pred_wrapper))
print("包裹法 — 分类报告:\n", classification_report(y_test, y_pred_wrapper, zero_division=0))
print("------------------------------------------------------------")
# 嵌入法s
clf_embedded = SVC(kernel='poly', C=1)
clf_embedded.fit(X_train_new_embedded, y_train)
y_pred_embedded = clf_embedded.predict(X_test_new_embedded)
print("嵌入法 — 准确率:", accuracy_score(y_test, y_pred_embedded))
print("嵌入法 — 分类报告:\n", classification_report(y_test, y_pred_embedded, zero_division=0))
print("-----------------------------------------------------------")

过滤法 — 准确率: 0.9629629629629629
过滤法 — 分类报告:
               precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       0.94      0.94      0.94        17
           2       1.00      0.94      0.97        17

    accuracy                           0.96        54
   macro avg       0.96      0.96      0.96        54
weighted avg       0.96      0.96      0.96        54

------------------------------------------------------------
包裹法 — 准确率: 0.8518518518518519
包裹法 — 分类报告:
               precision    recall  f1-score   support

           0       0.82      0.90      0.86        20
           1       0.76      0.76      0.76        17
           2       1.00      0.88      0.94        17

    accuracy                           0.85        54
   macro avg       0.86      0.85      0.85        54
weighted avg       0.86      0.85      0.85        54

------------------------------------------------------------
嵌入法 — 准确率: 0.6481481481481481
